In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import _pickle as cPickle
from collections import defaultdict
import torch
import torch.nn.init
from torch.autograd import Variable

In [2]:
dirname = './data_preprocessed_python'
filenames = os.listdir(dirname)
data_dic = defaultdict(lambda:[])
for i, filename in enumerate(filenames):
    full_filename = os.path.join(dirname, filename)
    x = cPickle.load(open(full_filename, 'rb'), encoding='ISO-8859-1')
    data_dic[filename[:-4]] = x

In [3]:
from scipy.stats import kurtosis, skew
def preprocessing(data):
    result = []
    mean = np.mean(data)
    median = np.median(data)
    maximum = np.amax(data)
    minimum = np.amin(data)
    std_dev = np.std(data)
    var = np.var(data)
    ran = np.ptp(data)
    skewness = skew(data)
    kurto = kurtosis(data)
    result=[mean, median, maximum, minimum,
                   std_dev, var, ran, skewness, kurto]
    return result

### make features and reform to use comfortably

In [4]:
new_data_dic = defaultdict(lambda:defaultdict(lambda:[]))
for key in data_dic.keys():
    par = int(key[1:])
    for i, vid in enumerate(data_dic[key]['data']):
        new_data_dic[key]['data'].append([])
        for chnn in vid[:-8]:
            l =preprocessing(np.array(chnn))
            for batch in range(0,10):
                l = l+preprocessing(np.array(chnn[batch*807:max((batch+1)*807, 8064)]))
            new_data_dic[key]['data'][i].append(np.array([par, i+1]+l))
    for i,  vid in enumerate(data_dic[key]['labels']):
        l2 = []
        l3 = []
        new_data_dic[key]['labels'].append([])
        new_data_dic[key]['labels2'].append([])
        for value in vid:
            if value>=5:
                l2.append(1)
                l2.append(0)
            else:
                l2.append(0)
                l2.append(1)
            if value>=6:
                l3.append(1)
                l3.append(0)
                l3.append(0)
            elif value>=4 and value<6:
                l3.append(0)
                l3.append(1)
                l3.append(0)
            else:
                l3.append(0)
                l3.append(0)
                l3.append(1)
        new_data_dic[key]['labels'][i] += l2
        new_data_dic[key]['labels2'][i] += l3

In [34]:
data_loader = []
test_loader = []
idx = 0
print(len(new_data_dic.keys()))
for par in new_data_dic.keys():
    if par != 's02':
        for i, vid in enumerate(new_data_dic[par]['data']):
            data_loader.append((np.array(vid), new_data_dic[par]['labels'][i]))
            idx +=1
    else:
        for i, vid in enumerate(new_data_dic[par]['data']):
            test_loader.append((np.array(vid), new_data_dic[par]['labels'][i]))
            idx +=1
print(len(data_loader))
print(data_loader[1])

32
1240
(array([[  1.00000000e+00,   2.00000000e+00,  -8.27121269e-02, ...,
          3.70042683e+01,  -5.08154218e-02,   8.69235014e-01],
       [  1.00000000e+00,   2.00000000e+00,  -3.13505384e-02, ...,
          4.18200988e+01,   1.11964916e-01,   1.39442973e+00],
       [  1.00000000e+00,   2.00000000e+00,  -2.04664705e-02, ...,
          3.87172925e+01,   5.88444745e-02,   9.21659624e-01],
       ..., 
       [  1.00000000e+00,   2.00000000e+00,   6.82457494e-02, ...,
          2.08783110e+01,   1.04035679e-01,   1.35067118e-03],
       [  1.00000000e+00,   2.00000000e+00,   1.42352938e-01, ...,
          5.25603937e+01,   1.71642208e-01,   2.84520300e+00],
       [  1.00000000e+00,   2.00000000e+00,   1.56159050e-01, ...,
          3.58672933e+01,   3.79596399e-01,   9.76703888e-01]]), [1, 0, 1, 0, 1, 0, 1, 0])


In [35]:
data_loader2 = []
test_loader2 = []
trX = []
trY = []
teX = []
teY = []
idx = 0
print(len(new_data_dic.keys()))
for par in new_data_dic.keys():
    if par != 's02':
        for i, vid in enumerate(new_data_dic[par]['data']):
            data_loader2.append((np.array(vid), new_data_dic[par]['labels2'][i]))
            idx +=1
    else:
        for i, vid in enumerate(new_data_dic[par]['data']):
            test_loader2.append((np.array(vid), new_data_dic[par]['labels2'][i]))
            idx +=1
print(len(data_loader2))
for i in data_loader:
    trX.append(i[0])
    trY.append(i[1][:2])
for i in test_loader:
    teX.append(i[0])
    teY.append(i[1][:2])
print(len(teY))

32
1240
40


## train, test - arousal 2 class

In [38]:
trX = []
trY = []
teX = []
teY = []
for i in data_loader:
    trX.append(i[0])
    trY.append(i[1][2:4])
for i in test_loader:
    teX.append(i[0])
    teY.append(i[1][2:4])

X = tf.placeholder("float", [None, 32, 101, 1])

Y = tf.placeholder("float", [None, 2])

trX = torch.Tensor(np.array(trX).reshape(-1, 32, 101, 1))
trY = np.array(trY)

training_epochs = 20

w = init_weights([101, 32, 1, 32])       # 101x32x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([6656, 625]) # FC 6656 inputs, 625 outputs
w_o = init_weights([625, 2])         # FC 625 inputs, 2 outputs (labels)

p_keep_conv = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)


py_x = model(trX, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=trY))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

init = tf.global_variables_initializer()

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    start_time = time.time()
    sess.run(init)

    for i in range(training_epochs):
        avg_cost = 0.
        avg_training_accuracy = 0.
        
        #not using batch
        batch_xs = trX
        batch_ys = trY

        batch_xs_image = torch.Tensor(np.array(batch_xs).reshape(-1, 32, 101, 1))
        batch_ys_image = np.array(batch_ys)

        sess.run(train_op, feed_dict={X: batch_xs_image, Y: batch_ys_image,
                                      p_keep_conv: 0.8, p_keep_hidden: 0.5})
        # Training average cost
        avg_cost += sess.run(cost, feed_dict={X: batch_xs_image, Y: batch_ys, p_keep_conv:1.0, p_keep_hidden:1.0})
        avg_training_accuracy += (np.mean(np.argmax(batch_ys, axis=1) ==
                     sess.run(predict_op, feed_dict={X: batch_xs_image,
                                                    Y: batch_ys_image,
                                                    p_keep_conv: 1.0,
                                                    p_keep_hidden: 1.0})))
            
        print("Epoch: %d, training error: %.4f, training accuracy: %.4f"%(i,avg_cost,avg_training_accuracy))

        # testing accuracy
        teX = torch.Tensor(np.array(teX).reshape(-1, 32, 101, 1))
        teY = np.array(teY)
        outputs = sess.run(predict_op, feed_dict={X: teX,
                                                        Y: teY,
                                                        p_keep_conv: 1.0,
                                                        p_keep_hidden: 1.0})

        cnt = 0
        for idx, out in enumerate(outputs[:40]):
            if teY[idx][out] == 1:
                cnt+=1
                
#         testing_accuracy = np.mean(np.argmax(teY, axis=1) ==
#                          sess.run(predict_op, feed_dict={X: teX,
#                                                         Y: teY,
#                                                         p_keep_conv: 1.0,
#                                                         p_keep_hidden: 1.0}))
        print("Testing Accuracy: %.4f"%(cnt/40))
        
        if testing_accuracy == 1.0:
            print("Early stop")
            break
        
    print("time elapsed: {:.2f}s".format(time.time() - start_time))

Epoch: 0, training error: 1026646.1875, training accuracy: 0.4113
Testing Accuracy: 0.4000
Epoch: 1, training error: 99390.5938, training accuracy: 0.4113
Testing Accuracy: 0.4000
Epoch: 2, training error: 3862.2773, training accuracy: 0.4484
Testing Accuracy: 0.4000
Epoch: 3, training error: 50157.6055, training accuracy: 0.5887
Testing Accuracy: 0.6000
Epoch: 4, training error: 7296.2231, training accuracy: 0.4137
Testing Accuracy: 0.4000
Epoch: 5, training error: 23503.0293, training accuracy: 0.5887
Testing Accuracy: 0.6000
Epoch: 6, training error: 2944.3008, training accuracy: 0.5081
Testing Accuracy: 0.4000
Epoch: 7, training error: 19463.2578, training accuracy: 0.5887
Testing Accuracy: 0.6000
Epoch: 8, training error: 2847.5857, training accuracy: 0.5161
Testing Accuracy: 0.4000
Epoch: 9, training error: 15236.6094, training accuracy: 0.5887
Testing Accuracy: 0.6000
Epoch: 10, training error: 2474.8794, training accuracy: 0.5685
Testing Accuracy: 0.4250
Epoch: 11, training err

## train, test - arousal 3 class

In [39]:
trX = []
trY = []
teX = []
teY = []
for i in data_loader2:
    trX.append(i[0])
    trY.append(i[1][3:6])
for i in test_loader2:
    teX.append(i[0])
    teY.append(i[1][3:6])

X = tf.placeholder("float", [None, 32, 101, 1])

Y = tf.placeholder("float", [None, 3])

trX = torch.Tensor(np.array(trX).reshape(-1, 32, 101, 1))
trY = np.array(trY)
    
w = init_weights([101, 32, 1, 32])       # 101x32x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([6656, 625]) # FC 6656 inputs, 625 outputs
w_o = init_weights([625, 3])         # FC 625 inputs, 2 outputs (labels)

p_keep_conv = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)


py_x = model(trX, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=trY))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

init = tf.global_variables_initializer()

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    start_time = time.time()
    sess.run(init)

    for i in range(training_epochs):
        avg_cost = 0.
        avg_training_accuracy = 0.
        
        #not using batch
        batch_xs = trX
        batch_ys = trY

        batch_xs_image = torch.Tensor(np.array(batch_xs).reshape(-1, 32, 101, 1))
        batch_ys_image = np.array(batch_ys)

        sess.run(train_op, feed_dict={X: batch_xs_image, Y: batch_ys_image,
                                      p_keep_conv: 0.8, p_keep_hidden: 0.5})
        # Training average cost
        avg_cost += sess.run(cost, feed_dict={X: batch_xs_image, Y: batch_ys, p_keep_conv:1.0, p_keep_hidden:1.0})
        avg_training_accuracy += (np.mean(np.argmax(batch_ys, axis=1) ==
                     sess.run(predict_op, feed_dict={X: batch_xs_image,
                                                    Y: batch_ys_image,
                                                    p_keep_conv: 1.0,
                                                    p_keep_hidden: 1.0})))
            
        print("Epoch: %d, training error: %.4f, training accuracy: %.4f"%(i,avg_cost,avg_training_accuracy))

        # testing accuracy
        teX = torch.Tensor(np.array(teX).reshape(-1, 32, 101, 1))
        teY = np.array(teY)
        outputs = sess.run(predict_op, feed_dict={X: teX,
                                                        Y: teY,
                                                        p_keep_conv: 1.0,
                                                        p_keep_hidden: 1.0})

        cnt = 0
        for idx, out in enumerate(outputs[:40]):
            if teY[idx][out] == 1:
                cnt+=1
                
#         testing_accuracy = np.mean(np.argmax(teY, axis=1) ==
#                          sess.run(predict_op, feed_dict={X: teX,
#                                                         Y: teY,
#                                                         p_keep_conv: 1.0,
#                                                         p_keep_hidden: 1.0}))
        print("Testing Accuracy: %.4f"%(cnt/40))
        
        if testing_accuracy == 1.0:
            print("Early stop")
            break
        
    print("time elapsed: {:.2f}s".format(time.time() - start_time))

Epoch: 0, training error: 715273.6250, training accuracy: 0.2863
Testing Accuracy: 0.3250
Epoch: 1, training error: 186929.0312, training accuracy: 0.4121
Testing Accuracy: 0.5250
Epoch: 2, training error: 36115.8242, training accuracy: 0.3815
Testing Accuracy: 0.3500
Epoch: 3, training error: 30610.7676, training accuracy: 0.2863
Testing Accuracy: 0.3250
Epoch: 4, training error: 15073.0020, training accuracy: 0.4121
Testing Accuracy: 0.5250
Epoch: 5, training error: 12053.9277, training accuracy: 0.2887
Testing Accuracy: 0.3250
Epoch: 6, training error: 10629.0625, training accuracy: 0.3363
Testing Accuracy: 0.1750
Epoch: 7, training error: 2975.2471, training accuracy: 0.3597
Testing Accuracy: 0.4000
Epoch: 8, training error: 8648.0723, training accuracy: 0.3121
Testing Accuracy: 0.1500
Epoch: 9, training error: 3031.4395, training accuracy: 0.4339
Testing Accuracy: 0.5250
Epoch: 10, training error: 5594.1826, training accuracy: 0.3073
Testing Accuracy: 0.1500
Epoch: 11, training er

#### I don't know why all the predicted labels are the same....
We tried several methods using smaller training set(size - 40) but we cannot fix it.
#### So we use another library(PyTorch) and other algorithms

## train, test - valence 2 class

In [32]:
trX = []
trY = []
teX = []
teY = []
training_epochs = 20
for i in data_loader:
    trX.append(i[0])
    trY.append(i[1][:2])
for i in test_loader:
    teX.append(i[0])
    teY.append(i[1][:2])
print(len(teY))

# Read data

X = tf.placeholder("float", [None, 32, 101, 1])

Y = tf.placeholder("float", [None, 2])

trX = torch.Tensor(np.array(trX[:40]).reshape(-1, 32, 101, 1))
trY = np.array(trY[:40])

w = init_weights([101, 32, 1, 32])       # 101x32x1 conv, 32 outputs
w2 = init_weights([101, 32, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([101, 32, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([6656, 625]) # FC 6656 inputs, 625 outputs
w_o = init_weights([625, 2])         # FC 625 inputs, 2 outputs (labels)

p_keep_conv = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)


py_x = model(trX, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=trY))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

init = tf.global_variables_initializer()

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    start_time = time.time()
    sess.run(init)

    for i in range(training_epochs):
        avg_cost = 0.
        avg_training_accuracy = 0.
        
        #not using batch
        batch_xs = trX
        batch_ys = trY

        batch_xs_image = torch.Tensor(np.array(batch_xs).reshape(-1, 32, 101, 1))
        batch_ys_image = np.array(batch_ys)

        sess.run(train_op, feed_dict={X: batch_xs_image, Y: batch_ys_image,
                                      p_keep_conv: 0.8, p_keep_hidden: 0.5})
        # Training average cost
        avg_cost += sess.run(cost, feed_dict={X: batch_xs_image, Y: batch_ys, p_keep_conv:1.0, p_keep_hidden:1.0})
        avg_training_accuracy += (np.mean(np.argmax(batch_ys, axis=1) ==
                     sess.run(predict_op, feed_dict={X: batch_xs_image,
                                                    Y: batch_ys_image,
                                                    p_keep_conv: 1.0,
                                                    p_keep_hidden: 1.0})))
            
        print("Epoch: %d, training error: %.4f, training accuracy: %.4f"%(i,avg_cost,avg_training_accuracy))

        # testing accuracy
        teX = torch.Tensor(np.array(teX).reshape(-1, 32, 101, 1))
        teY = np.array(teY)

        testing_accuracy = np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX,
                                                        Y: teY,
                                                        p_keep_conv: 1.0,
                                                        p_keep_hidden: 1.0}))
        print("Testing Accuracy: %.4f"%(testing_accuracy))
        
        if testing_accuracy == 1.0:
            print("Early stop")
            break
        
    print("time elapsed: {:.2f}s".format(time.time() - start_time))

40
Epoch: 0, training error: 1734375.6250, training accuracy: 0.5250
Testing Accuracy: 0.3750
Epoch: 1, training error: 32420.9785, training accuracy: 0.4750
Testing Accuracy: 0.6250
Epoch: 2, training error: 11913.7402, training accuracy: 0.5250
Testing Accuracy: 0.3750
Epoch: 3, training error: 2246.6777, training accuracy: 0.4750
Testing Accuracy: 0.6250
Epoch: 4, training error: 9822.4473, training accuracy: 0.5250
Testing Accuracy: 0.3750
time elapsed: 510.71s
